In [1]:
library(readxl)
library(tidyr)
library(tidyverse)
library(caret)

# Suppress warnings globally
options(warn = -1)

# set working directory
# setwd('/lustre/home/reynaj/Projects/20241011.Byrd_Lab.IBD_NuLisa')
setwd('/home/reynaj/projects/kevin_byrd/20241011.kevin_byrd.ibd_nulisa')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.0.4
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.2     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




## Process the protein levels

In [2]:
# load assay data
fn <- "results/raw/P-000458_ADA_NULISAseq_Inflammation Panel_1-NPQ Counts_2-Target Detectability_3-Sample Information_2024_08_26.xlsx"
data <- read_excel(fn, sheet = "NPQ Counts");

In [3]:
data[1:3,]

Panel,PanelLotNumber,PlateID,SampleName,SampleType,Target,AlamarTargetID,UniProtID,ProteinName,SampleQC,LOD,NPQ
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
200-Plex Inflammation Panel v2,panelLot022,Plate_01,A_01_TRP-34082,Sample,AGER,t5521,Q15109,Advanced glycosylation end product-specific receptor,PASS,0,5.493463
200-Plex Inflammation Panel v2,panelLot022,Plate_01,A_02_TRP-34086,Sample,AGER,t5521,Q15109,Advanced glycosylation end product-specific receptor,PASS,0,4.453723
200-Plex Inflammation Panel v2,panelLot022,Plate_01,A_03_TRP-42335,Sample,AGER,t5521,Q15109,Advanced glycosylation end product-specific receptor,PASS,0,6.469100


In [4]:
# remove the control samples
pd_data <- data %>% filter(!SampleName %in% c('A_12_SC_Rep01', 'B_12_SC_Rep02', 'C_12_SC_Rep03'))

# generate indicator of whether NPQ < LOD
pd_data <- pd_data %>% mutate(NPQ_below_LOD = NPQ < LOD)

# calculate the number of samples
num_samples <- nrow(unique(pd_data[,'SampleName']))

protein_detectability <- pd_data %>% group_by(Target) %>%
                            summarise(portion_NPQ_below_LOD = sum(NPQ_below_LOD) / num_samples) %>%
                            arrange(desc(portion_NPQ_below_LOD))

low_protein_detectability <- protein_detectability[protein_detectability[,'portion_NPQ_below_LOD'] >= 0.5,] %>% drop_na()
high_protein_detectability <- protein_detectability[protein_detectability[,'portion_NPQ_below_LOD'] < 0.5,] %>% drop_na()

In [5]:
nrow(high_protein_detectability)

[1] 230

In [6]:
# filter for only good proteins
filtered_data <- data %>% filter(Target %in% high_protein_detectability$Target)

# pivot the data to make a matrix
wide_data <- filtered_data %>% pivot_wider(id_cols = SampleName, names_from = Target, values_from = NPQ)

In [7]:
# transpose the data to be in the correct format for limma
# rows = genes
# columns = samples
t_data = t(wide_data)

# clean up the first row
colnames(t_data) <- t_data[1,]
t_data <- as.data.frame(t_data[-1,])

# get the column names
sample_names <- colnames(t_data)

# remove the pattern [A-Z]_[0-9]+
new_sample_names <- gsub("[A-Z]_[0-9]+_", "", sample_names)

# assign the new column names back to the data frame
colnames(t_data) <- new_sample_names

In [8]:

# # check near zero variance # didn't find any zeroVar nor nzv
nzv_info <- nearZeroVar(t(t_data), saveMetrics = TRUE)
lapply(nzv_info, sum)


$freqRatio
[1] 1562

$percentUnique
[1] 21410.11

$zeroVar
[1] 0

$nzv
[1] 0

In [9]:
write.table(t_data, "results/specimen_focused/comp_data/protein_levels.npq.tsv", sep = "\t", col.names = TRUE, quote=FALSE)

## Process the clinical data

In [10]:
# load clinical data
fn <- "results/raw/ADA_IBD_Saliva_Biospecimen Manifest Form-for-NULISA_241022.V2.Updated_Sample20.xlsx"
clinical_data <- read_excel(fn, sheet = "Aliquot Information")

In [11]:
# rename columns for programming use
transform_string <- function(x) {
  x <- tolower(x)            # Convert to lowercase
  x <- gsub(" ", "_", x)     # Replace spaces with "_"
  x <- gsub("\\(", "_", x)   # Replace "(" with "_"
  x <- gsub("\\)", "", x)    # Remove ")"
  return(x)
}

# Apply the function to the vector
transformed_cols <- sapply(as.vector(colnames(clinical_data)), transform_string)
colnames(clinical_data) <- transformed_cols

In [12]:
# remove nan samples
clinical_data <- clinical_data[!is.na(clinical_data$project_name),]
clinical_sample_names = clinical_data$original_subject_id

In [13]:
# Create a new row as a dataframe
new_row <- data.frame(
  original_subject_id = c("SC_Rep01", "SC_Rep02", "SC_Rep03"),
  sample_id = c("SC_Rep01", "SC_Rep02", "SC_Rep03"),
  ibd_diagnosis = c("Alamar_Sample_Control", "Alamar_Sample_Control", "Alamar_Sample_Control"),
  disease_activity = c("N/A", "N/A", "N/A")
)

In [14]:
# Add the new row to the dataframe
clinical_data <- bind_rows(clinical_data, new_row)

# order the dataframe based on the order of new sample names
clinical_data <- clinical_data[match(new_sample_names, clinical_data$original_subject_id), ]

In [15]:
# def indicator function
get_indicator <- function(x, check_list, categories) {
  
  if (x %in% check_list) {
    return(categories[[1]])
  } 
  else {
    return(categories[[2]])
  }
}

# add ibd disease indicator
ibd_check_list = c("CD", "UC", "IBD-U")
ibd_indicator <- sapply(as.vector(clinical_data$ibd_diagnosis), get_indicator, check_list=ibd_check_list, categories=c("IBD Super Group", "Control Super Group"))
clinical_data$ibd_indicator <- ibd_indicator

# add disease activity indicator
da_check_list = c("Moderate", "Mild")
da_indicator <- sapply(as.vector(clinical_data$disease_activity), get_indicator, check_list=da_check_list, categories=c("Active Disease", "In-active Disease"))
clinical_data$disease_activity_indicator <- da_indicator

### Add the CRP levels

In [16]:
# load the data
crp_fn = "results/raw/IBDSCANInflammatoryB-CRPForSeverity_DATA_2024-12-02_1231.csv"
crp_df <- read_csv(crp_fn)

# make the matched_subject column to match the clinical data
crp_df <- crp_df %>% mutate(matched_subject_id=as.integer(sub("^0+", "", subject_id)))

# aggregate the crp_values
agg_crp_df <- crp_df %>% group_by(matched_subject_id) %>% summarize(max_crp_value=max(crp_value, np.rm=TRUE))

# get a crp_label using rubric by Kevin
# 1. Normal CRP Levels -- <1 mg/L: Indicates a low level of inflammation and is considered normal in healthy individuals.
# 2. Mild Elevation -- 1–3 mg/L: May suggest low-grade inflammation, often associated with chronic conditions such as mild infection.
# 3. Moderate Elevation-- 3–10 mg/L: Indicates moderate inflammation. signifies chronic inflammation or acute infections (e.g., respiratory infections).
# 4. High Elevation -- >10 mg/L: Suggests significant inflammation or infection.
get_crp_severity_label <- function(x){
    if (is.na(x)){
        return("N/A")
    } else if (x < 1){
        return("Normal")
    } else if (1 <= x  & x < 3){
        return("Mild")
    } else if (3 <= x & x <10){
        return("Moderate")
    } else {
        return("High")
    }

}
#agg_crp_df %>% mutate(crp_label=get_crp_label(max_crp_value))
agg_crp_df$crp_severity <- vapply(agg_crp_df$max_crp_value, get_crp_severity_label,FUN.VALUE="test")

Rows: 106 Columns: 9
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): subject_id, redcap_event_name
dbl  (5): crp_days, crp_value, crp_30d_yn, ibd_diagnosis, ibd_disease_activity
date (2): crp_date, saliva_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [17]:
clinical_data <- clinical_data %>% left_join(agg_crp_df,by = "matched_subject_id")
clinical_data$crp_severity <- replace_na(clinical_data$crp_severity, "N/A")

In [18]:
write.table(clinical_data, "results/specimen_focused/comp_data/clinical_data.tsv", sep = "\t", col.names = TRUE, row.names=FALSE, quote=FALSE)